In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f5987c67-0e50-4099-960d-33ba7aa51e7c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 254ms :: artifacts dl 15ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

24/02/22 14:22:45 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
from delta.tables import *

In [5]:
DeltaTable.createIfNotExists(sparkSession=spark).tableName("employee_demo").addColumn(
    "emp_id", "INT"
).addColumn("emp_name", "STRING").addColumn("gender", "STRING").addColumn(
    "salary", "INT"
).addColumn(
    "Dept", "STRING"
).property(
    "description", "table created for demo purpose"
).location(
    "/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_07"
).execute()

In [6]:
spark.sql("select * from employee_demo").show(truncate=False)

+------+--------+------+------+----+
|emp_id|emp_name|gender|salary|Dept|
+------+--------+------+------+----+
+------+--------+------+------+----+



In [7]:
spark.sql("INSERT INTO employee_demo VALUES(10,'Stephen','M',2000,'IT');")

DataFrame[]

In [8]:
spark.sql("select * from employee_demo").show(truncate=False)

+------+--------+------+------+----+
|emp_id|emp_name|gender|salary|Dept|
+------+--------+------+------+----+
|10    |Stephen |M     |2000  |IT  |
+------+--------+------+------+----+



### Schema Evaluation

In [9]:
from pyspark.sql.types import IntegerType, StringType

In [12]:
employee_data = [(200, "Philipp", "M", 8000, "HR", "test data")]

employee_schema = StructType(
    [
        StructField("emp_id", IntegerType(), False),
        StructField("emp_name", StringType(), False),
        StructField("gender", StringType(), False),
        StructField("salary", IntegerType(), False),
        StructField("dept", StringType(), False),
        StructField("additoinalcolumn1", StringType(), False),
    ]
)

df = spark.createDataFrame(data=employee_data, schema=employee_schema)
df.show(truncate=False)

+------+--------+------+------+----+-----------------+
|emp_id|emp_name|gender|salary|dept|additoinalcolumn1|
+------+--------+------+------+----+-----------------+
|200   |Philipp |M     |8000  |HR  |test data        |
+------+--------+------+------+----+-----------------+



In [13]:
df.write.format("delta").mode("append").saveAsTable("employee_demo")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 6b5dacec-20ee-49ee-8cc1-f0128ce98239).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- emp_id: integer (nullable = true)
-- emp_name: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)
-- Dept: string (nullable = true)


Data schema:
root
-- emp_id: integer (nullable = true)
-- emp_name: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)
-- dept: string (nullable = true)
-- additoinalcolumn1: string (nullable = true)

         

- To handle the error

In [14]:
df.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable(
    "employee_demo"
)

In [15]:
spark.sql("select * from employee_demo").show(truncate=False)

+------+--------+------+------+----+-----------------+
|emp_id|emp_name|gender|salary|Dept|additoinalcolumn1|
+------+--------+------+------+----+-----------------+
|200   |Philipp |M     |8000  |HR  |test data        |
|10    |Stephen |M     |2000  |IT  |NULL             |
+------+--------+------+------+----+-----------------+



In [16]:
employee_data = [(300, "David", "M", 8000, "HR", "dummy data")]

employee_schema = StructType(
    [
        StructField("emp_id", IntegerType(), False),
        StructField("emp_name", StringType(), False),
        StructField("gender", StringType(), False),
        StructField("salary", IntegerType(), False),
        StructField("dept", StringType(), False),
        StructField("additoinalcolumn2", StringType(), False),
    ]
)

df = spark.createDataFrame(data=employee_data, schema=employee_schema)
df.show(truncate=False)

+------+--------+------+------+----+-----------------+
|emp_id|emp_name|gender|salary|dept|additoinalcolumn2|
+------+--------+------+------+----+-----------------+
|300   |David   |M     |8000  |HR  |dummy data       |
+------+--------+------+------+----+-----------------+



In [17]:
df.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable(
    "employee_demo"
)

In [18]:
spark.sql("select * from employee_demo").show(truncate=False)

+------+--------+------+------+----+-----------------+-----------------+
|emp_id|emp_name|gender|salary|Dept|additoinalcolumn1|additoinalcolumn2|
+------+--------+------+------+----+-----------------+-----------------+
|300   |David   |M     |8000  |HR  |NULL             |dummy data       |
|200   |Philipp |M     |8000  |HR  |test data        |NULL             |
|10    |Stephen |M     |2000  |IT  |NULL             |NULL             |
+------+--------+------+------+----+-----------------+-----------------+

